In [16]:
import spacy

nlp = spacy.load('de_core_news_sm')
doc = nlp("Entwickler") 
for token in doc:
     print(token, token.lemma_, token.morph.get("Gender")) 
doc = nlp("Entwicklerin") 
for token in doc:
     print(token, token.lemma_, token.morph.get("Gender"))   

Reinigungskraft Reinigungskraft ['Fem']
Entwicklerin Entwicklerin ['Fem']
